In [1]:

import models
import utils
import data_processing
from constants import (
    VALUE_COLUMN,
    UNIT_RATE_COLUMN,
    QUANTITY_COLUMN,
    GROSS_WEIGHT_COLUMN,
    SPIKES_WINDOW_SIZE,
    SHIP_COUNT_COLUMN,
    PORT_COUNT_COLUMN,
    FILL_METHOD,
    TARGET_COLUMN
)
import pandas as pd
import constants
import sys
from datetime import datetime
from build_data import get_data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer

In [2]:
COMMODITY = "magnesium"

VOLZA_FILE_PATH = f"../volza/{COMMODITY}/{COMMODITY}.csv"
PRICE_FILE_PATH = f"../volza/{COMMODITY}/{COMMODITY}_prices.csv"

# Get the data
data = get_data(VOLZA_FILE_PATH, PRICE_FILE_PATH)

/Users/harshdeepsingh/ASU/Lab_V2/GSN/onr_price_prediction/analysis/build_data.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brent_df.rename(columns={"Value": "Brent Oil Value"}, inplace=True)
/Users/harshdeepsingh/ASU/Lab_V2/GSN/onr_price_prediction/analysis/build_data.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wti_df.rename(columns={"Value": "WTI Oil Value"}, inplace=True)


In [4]:
# Trying out different window sizes
SPIKE_WINDOW_SIZES = [10, 20, 30, 40, 60, 80, 100]
results_dfs = []

for window_size in SPIKE_WINDOW_SIZES:
    aggregated_df = data.copy()
    SPIKES_WINDOW_SIZE = window_size
    VOLZA_FEATURES = [VALUE_COLUMN, UNIT_RATE_COLUMN, QUANTITY_COLUMN, GROSS_WEIGHT_COLUMN]
    X, y = data_processing.prepare_features_and_target(aggregated_df, VOLZA_FEATURES, 'spikes')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    X_train, X_test = data_processing.scale_features(X_train, X_test)

    X_train, y_train = data_processing.create_sequences(X_train, y_train, SPIKES_WINDOW_SIZE)
    X_test, y_test = data_processing.create_sequences(X_test, y_test, SPIKES_WINDOW_SIZE)
    
    print(f'X train shape: {X_train.shape}')

    output_file_path = f'{COMMODITY}/volza/results_{window_size}.csv'
    pred_file_path = f'{COMMODITY}/volza/predictions/{window_size}'
    print(pred_file_path)
    results_df = models.evaluate_all(X_train, y_train, X_test, y_test, output_file_path, pred_file_path)
    results_df['Window Size'] = window_size
    results_dfs.append(results_df)
    

X train shape: (766, 10, 4)
magnesium/volza/predictions/10
6/6 [==============================] - 0s 3ms/step
Predictions saved to CSV file: magnesium/volza/predictions/10/LSTM_250_layers_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/10/LSTM_250_layers_predictions.npy
6/6 [==============================] - 0s 2ms/step
Predictions saved to CSV file: magnesium/volza/predictions/10/LSTM_200_layers_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/10/LSTM_200_layers_predictions.npy
6/6 [==============================] - 0s 2ms/step
Predictions saved to CSV file: magnesium/volza/predictions/10/LSTM_100_layers_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/10/LSTM_100_layers_predictions.npy
6/6 [==============================] - 0s 1ms/step
Predictions saved to CSV file: magnesium/volza/predictions/10/LSTM_50_layers_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/10/LSTM_50_layers_pre

In [144]:
#Volza Only Features
aggregated_df = data.copy()
VOLZA_FEATURES = [VALUE_COLUMN, UNIT_RATE_COLUMN, QUANTITY_COLUMN, GROSS_WEIGHT_COLUMN]
X, y = data_processing.prepare_features_and_target(aggregated_df, VOLZA_FEATURES, 'spikes')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_test = data_processing.scale_features(X_train, X_test)

X_train, y_train = data_processing.create_sequences(X_train, y_train, SPIKES_WINDOW_SIZE)
X_test, y_test = data_processing.create_sequences(X_test, y_test, SPIKES_WINDOW_SIZE)

output_file_path = f'{COMMODITY}_results_volza_only.csv'
models.evaluate_all(X_train, y_train, X_test, y_test, output_file_path)

8/8 [==============================] - 0s 726us/step


,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1)
0,LSTM,250 layers,0.799127,0.896552,0.879227,0.887805,0.038462,0.045455,0.041667
1,LSTM,200 layers,0.794760,0.904040,0.864734,0.883951,0.096774,0.136364,0.113208
2,LSTM,100 layers,0.799127,0.904523,0.869565,0.886700,0.100000,0.136364,0.115385
3,LSTM,50 layers,0.803493,0.900990,0.879227,0.889976,0.074074,0.090909,0.081633
4,CNN with Attention,"32 filters, kernel size 7",0.851528,0.902326,0.937198,0.919431,0.071429,0.045455,0.055556
5,CNN with Attention,"32 filters, kernel size 5",0.890830,0.902655,0.985507,0.942263,0.000000,0.000000,0.000000
6,CNN with Attention,"32 filters, kernel size 3",0.873362,0.904545,0.961353,0.932084,0.111111,0.045455,0.064516
7,CNN with Attention,"64 filters, kernel size 7",0.855895,0.902778,0.942029,0.921986,0.076923,0.045455,0.057143
8,CNN with Attention,"64 filters, kernel size 5",0.868996,0.904110,0.956522,0.929577,0.100000,0.045455,0.062500
9,CNN with Attention,"64 filters, kernel size 3",0.873362,0.908257,0.956522,0.931765,0.181818,0.090909,0.121212


In [145]:
#Volza and AIS Features
aggregated_df = data.copy()
VOLZA_AIS_FEATURES = [VALUE_COLUMN, UNIT_RATE_COLUMN, QUANTITY_COLUMN, GROSS_WEIGHT_COLUMN,SHIP_COUNT_COLUMN, PORT_COUNT_COLUMN ]
aggregated_df[VOLZA_AIS_FEATURES] = aggregated_df[VOLZA_AIS_FEATURES].fillna(0)

X, y = data_processing.prepare_features_and_target(aggregated_df, VOLZA_AIS_FEATURES, 'spikes')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_test = data_processing.scale_features(X_train, X_test)

X_train, y_train = data_processing.create_sequences(X_train, y_train, SPIKES_WINDOW_SIZE)
X_test, y_test = data_processing.create_sequences(X_test, y_test, SPIKES_WINDOW_SIZE)

output_file_path = f'{COMMODITY}_results_volza_ais.csv'
models.evaluate_all(X_train, y_train, X_test, y_test, output_file_path)

8/8 [==============================] - 0s 630us/step


,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1)
0,LSTM,250 layers,0.676856,0.913043,0.710145,0.798913,0.117647,0.363636,0.177778
1,LSTM,200 layers,0.742358,0.920455,0.782609,0.845953,0.150943,0.363636,0.213333
2,LSTM,100 layers,0.711790,0.927273,0.739130,0.822581,0.156250,0.454545,0.232558
3,LSTM,50 layers,0.825328,0.907317,0.898551,0.902913,0.125000,0.136364,0.130435
4,CNN with Attention,"32 filters, kernel size 7",0.707424,0.916667,0.743961,0.821333,0.131148,0.363636,0.192771
5,CNN with Attention,"32 filters, kernel size 5",0.751092,0.912088,0.801932,0.853470,0.127660,0.272727,0.173913
6,CNN with Attention,"32 filters, kernel size 3",0.838428,0.908654,0.913043,0.910843,0.142857,0.136364,0.139535
7,CNN with Attention,"64 filters, kernel size 7",0.676856,0.913043,0.710145,0.798913,0.117647,0.363636,0.177778
8,CNN with Attention,"64 filters, kernel size 5",0.838428,0.904762,0.917874,0.911271,0.105263,0.090909,0.097561
9,CNN with Attention,"64 filters, kernel size 3",0.681223,0.918750,0.710145,0.801090,0.130435,0.409091,0.197802


In [146]:
#AIS Features
aggregated_df = data.copy()
AIS_FEATURES = [SHIP_COUNT_COLUMN, PORT_COUNT_COLUMN]
# aggregated_df = aggregated_df.dropna(subset=[SHIP_COUNT_COLUMN, PORT_COUNT_COLUMN])
aggregated_df[AIS_FEATURES] = aggregated_df[AIS_FEATURES].fillna(method=FILL_METHOD)
X, y = data_processing.prepare_features_and_target(aggregated_df, AIS_FEATURES, 'spikes')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_test = data_processing.scale_features(X_train, X_test)

X_train, y_train = data_processing.create_sequences(X_train, y_train, SPIKES_WINDOW_SIZE)
X_test, y_test = data_processing.create_sequences(X_test, y_test, SPIKES_WINDOW_SIZE)

# print(y_test)
output_file_path = f'{COMMODITY}_results_ais_only.csv'
models.evaluate_all(X_train, y_train, X_test, y_test, output_file_path)

8/8 [==============================] - 0s 4ms/step


/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

8/8 [==============================] - 0s 3ms/step


/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

8/8 [==============================] - 0s 2ms/step


...

8/8 [==============================] - 0s 1ms/step


/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

8/8 [==============================] - 0s 948us/step


/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1)
0,LSTM,250 layers,0.90393,0.90393,1.0,0.949541,0.0,0.0,0.0
1,LSTM,200 layers,0.90393,0.90393,1.0,0.949541,0.0,0.0,0.0
2,LSTM,100 layers,0.90393,0.90393,1.0,0.949541,0.0,0.0,0.0
3,LSTM,50 layers,0.90393,0.90393,1.0,0.949541,0.0,0.0,0.0
4,CNN with Attention,"32 filters, kernel size 7",0.90393,0.90393,1.0,0.949541,0.0,0.0,0.0
5,CNN with Attention,"32 filters, kernel size 5",0.90393,0.90393,1.0,0.949541,0.0,0.0,0.0
6,CNN with Attention,"32 filters, kernel size 3",0.90393,0.90393,1.0,0.949541,0.0,0.0,0.0
7,CNN with Attention,"64 filters, kernel size 7",0.90393,0.90393,1.0,0.949541,0.0,0.0,0.0
8,CNN with Attention,"64 filters, kernel size 5",0.90393,0.90393,1.0,0.949541,0.0,0.0,0.0
9,CNN with Attention,"64 filters, kernel size 3",0.90393,0.90393,1.0,0.949541,0.0,0.0,0.0


In [147]:
#AIS and Volza with Price ARIMA
import pmdarima as pm

aggregated_df = data.copy()

columns_of_interest = ['Price', 'spikes']  # Add other columns as necessary

aggregated_df = aggregated_df.dropna(subset=columns_of_interest)
# Fit an Auto ARIMA model to the 'Price' series
model = pm.auto_arima(aggregated_df['Price'], seasonal=True, m=12, suppress_warnings=True, stepwise=True, error_action='ignore')

# Forecast the series using the model (in-sample prediction)
forecast = model.predict_in_sample()

# Calculate residuals (difference between actual and forecasted values)
residuals = aggregated_df[TARGET_COLUMN] - forecast

# Append residuals to DataFrame as a new feature (using residuals as a way to detect spike / anomaly)
aggregated_df = aggregated_df.copy()
aggregated_df['ARIMA_Residuals'] = residuals

# Prepare features and target
FEATURE_COLUMNS = [TARGET_COLUMN,VALUE_COLUMN, UNIT_RATE_COLUMN, QUANTITY_COLUMN, GROSS_WEIGHT_COLUMN,SHIP_COUNT_COLUMN, PORT_COUNT_COLUMN, 'ARIMA_Residuals']  # Adjust as needed
aggregated_df[FEATURE_COLUMNS] = aggregated_df[FEATURE_COLUMNS].fillna(0)
X, y = data_processing.prepare_features_and_target(aggregated_df, FEATURE_COLUMNS, 'spikes')

# Split data (using sklearn's train_test_split)
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Scale features
X_train_scaled, X_test_scaled = data_processing.scale_features(X_train_raw, X_test_raw)

# Create sequences
X_train, y_train = data_processing.create_sequences(X_train_scaled, y_train, SPIKES_WINDOW_SIZE)
X_test, y_test = data_processing.create_sequences(X_test_scaled, y_test, SPIKES_WINDOW_SIZE)

# Evaluate all models & save in file
output_file_path = f'{COMMODITY}_results_volza_ais_arima.csv'
models.evaluate_all(X_train, y_train, X_test, y_test, output_file_path)

8/8 [==============================] - 0s 783us/step


,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1)
0,LSTM,250 layers,0.882096,0.924528,0.946860,0.935561,0.352941,0.272727,0.307692
1,LSTM,200 layers,0.864629,0.958333,0.888889,0.922306,0.378378,0.636364,0.474576
2,LSTM,100 layers,0.868996,0.931707,0.922705,0.927184,0.333333,0.363636,0.347826
3,LSTM,50 layers,0.834061,0.933333,0.879227,0.905473,0.264706,0.409091,0.321429
4,CNN with Attention,"32 filters, kernel size 7",0.877729,0.916279,0.951691,0.933649,0.285714,0.181818,0.222222
5,CNN with Attention,"32 filters, kernel size 5",0.847162,0.913462,0.917874,0.915663,0.190476,0.181818,0.186047
6,CNN with Attention,"32 filters, kernel size 3",0.882096,0.901786,0.975845,0.937355,0.000000,0.000000,0.000000
7,CNN with Attention,"64 filters, kernel size 7",0.829694,0.924242,0.884058,0.903704,0.225806,0.318182,0.264151
8,CNN with Attention,"64 filters, kernel size 5",0.882096,0.912844,0.961353,0.936471,0.272727,0.136364,0.181818
9,CNN with Attention,"64 filters, kernel size 3",0.877729,0.916279,0.951691,0.933649,0.285714,0.181818,0.222222


In [148]:
#Binarise the data
def discretize(df, columns, bins):
    est = KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy='kmeans')
    df[columns] = est.fit_transform(df[columns])
    return df

aggregated_df = data.copy()
VOLZA_AIS_FEATURES = [VALUE_COLUMN, UNIT_RATE_COLUMN, QUANTITY_COLUMN, GROSS_WEIGHT_COLUMN,SHIP_COUNT_COLUMN, PORT_COUNT_COLUMN ]
aggregated_df[VOLZA_AIS_FEATURES] = aggregated_df[VOLZA_AIS_FEATURES].fillna(0)
binary_columns = discretize(aggregated_df, VOLZA_AIS_FEATURES, 2)

X, y = data_processing.prepare_features_and_target(binary_columns, VOLZA_AIS_FEATURES, 'spikes')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_test = data_processing.scale_features(X_train, X_test)

X_train, y_train = data_processing.create_sequences(X_train, y_train, SPIKES_WINDOW_SIZE)
X_test, y_test = data_processing.create_sequences(X_test, y_test, SPIKES_WINDOW_SIZE)

output_file_path = f'{COMMODITY}_results_volza_ais_binary_med.csv'
models.evaluate_all(X_train, y_train, X_test, y_test, output_file_path)

8/8 [==============================] - 0s 634us/step


,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1)
0,LSTM,250 layers,0.676856,0.918239,0.705314,0.797814,0.128571,0.409091,0.195652
1,LSTM,200 layers,0.663755,0.916667,0.690821,0.787879,0.123288,0.409091,0.189474
2,LSTM,100 layers,0.803493,0.905000,0.874396,0.889435,0.103448,0.136364,0.117647
3,LSTM,50 layers,0.790393,0.899497,0.864734,0.881773,0.066667,0.090909,0.076923
4,CNN with Attention,"32 filters, kernel size 7",0.864629,0.907407,0.946860,0.926714,0.153846,0.090909,0.114286
5,CNN with Attention,"32 filters, kernel size 5",0.689956,0.904762,0.734300,0.810667,0.098361,0.272727,0.144578
6,CNN with Attention,"32 filters, kernel size 3",0.829694,0.900000,0.913043,0.906475,0.052632,0.045455,0.048780
7,CNN with Attention,"64 filters, kernel size 7",0.842795,0.905213,0.922705,0.913876,0.111111,0.090909,0.100000
8,CNN with Attention,"64 filters, kernel size 5",0.672489,0.907407,0.710145,0.796748,0.104478,0.318182,0.157303
9,CNN with Attention,"64 filters, kernel size 3",0.838428,0.904762,0.917874,0.911271,0.105263,0.090909,0.097561


# Try Spectral Residuals